In [12]:
import pandas as pd

In [13]:
# https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data
df = pd.read_csv('btcusd_1-min_data.csv')

# format timestamp
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
df.set_index('Timestamp', inplace=True)

print(df.shape)
df

(6747281, 5)


,Open,High,Low,Close,Volume
Timestamp,,,,,
2012-01-01 10:01:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:02:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:03:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:04:00,4.58,4.58,4.58,4.58,0.000000
2012-01-01 10:05:00,4.58,4.58,4.58,4.58,0.000000
...,...,...,...,...,...
2025-01-20 23:57:00,102202.00,102211.00,102068.00,102096.00,0.204232
2025-01-20 23:58:00,102043.00,102136.00,101999.00,102048.00,0.274637
2025-01-20 23:59:00,102047.00,102141.00,102019.00,102141.00,0.358309


### minute price data

In [14]:
df_minute = df.copy()

# calculate return (forward 1 min)
df_minute['return_forward'] = df_minute['Close'].pct_change().shift(-1)

# last row does not have return_forward
df_minute = df_minute.iloc[:-1]

df_minute = df_minute[df_minute.index.year < 2025]

# save as parquet
df_minute.to_parquet('btcusd_minute_price.parquet')

print(df_minute.shape)
df_minute

(6737279, 6)


,Open,High,Low,Close,Volume,return_forward
Timestamp,,,,,,
2012-01-01 10:01:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 10:02:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 10:03:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 10:04:00,4.58,4.58,4.58,4.58,0.000000,0.000000
2012-01-01 10:05:00,4.58,4.58,4.58,4.58,0.000000,0.000000
...,...,...,...,...,...,...
2024-12-31 23:55:00,93476.00,93476.00,93471.00,93471.00,0.179799,-0.000021
2024-12-31 23:56:00,93469.00,93469.00,93469.00,93469.00,0.021400,-0.000449
2024-12-31 23:57:00,93462.00,93462.00,93413.00,93427.00,0.034750,0.000000


In [15]:
# print rows per year
print(df_minute.resample('YE').size())

Timestamp
2012-12-31    526379
2013-12-31    525540
2014-12-31    525540
2015-12-31    525540
2016-12-31    526980
2017-12-31    525540
2018-12-31    525540
2019-12-31    525540
2020-12-31    526980
2021-12-31    525540
2022-12-31    525540
2023-12-31    525540
2024-12-31    427080
Freq: YE-DEC, dtype: int64


In [16]:
# check for missing minutes

# Generate a complete date range with minute frequency
full_range = pd.date_range(start=df_minute.index.min(), end=df_minute.index.max(), freq='T')

# Identify missing minutes
missing_minutes = full_range.difference(df_minute.index)

# print number of missing minutes in each year
print(missing_minutes.year.value_counts())

C:\Users\phku0\AppData\Local\Temp\ipykernel_22868\521674258.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=df_minute.index.min(), end=df_minute.index.max(), freq='T')


2024    99960
2013       60
2014       60
2015       60
2012       60
2016       60
2017       60
2019       60
2018       60
2020       60
2021       60
2022       60
2023       60
Name: count, dtype: int64


In [17]:
# missing minutes in 2024 only
missing_minutes[missing_minutes.year == 2024]

DatetimeIndex(['2024-06-01 00:41:00', '2024-06-01 00:42:00',
               '2024-06-01 00:43:00', '2024-06-01 00:44:00',
               '2024-06-01 00:45:00', '2024-06-01 00:46:00',
               '2024-06-01 00:47:00', '2024-06-01 00:48:00',
               '2024-06-01 00:49:00', '2024-06-01 00:50:00',
               ...
               '2024-12-31 07:11:00', '2024-12-31 07:12:00',
               '2024-12-31 07:13:00', '2024-12-31 07:14:00',
               '2024-12-31 07:15:00', '2024-12-31 07:16:00',
               '2024-12-31 07:17:00', '2024-12-31 07:18:00',
               '2024-12-31 07:19:00', '2024-12-31 07:20:00'],
              dtype='datetime64[ns]', length=99960, freq=None)

In [23]:
tmp = df_minute[df_minute.index > '2024-12-30']
tmp

,Open,High,Low,Close,Volume,return_forward
Timestamp,,,,,,
2024-12-31 07:21:00,92702.0,92709.0,92702.0,92709.0,0.326745,0.000431
2024-12-31 07:22:00,92724.0,92749.0,92724.0,92749.0,0.288580,0.000000
2024-12-31 07:23:00,92749.0,92749.0,92749.0,92749.0,0.000000,0.000000
2024-12-31 07:24:00,92749.0,92749.0,92749.0,92749.0,0.000000,-0.000011
2024-12-31 07:25:00,92757.0,92761.0,92745.0,92748.0,0.326955,-0.000173
...,...,...,...,...,...,...
2024-12-31 23:55:00,93476.0,93476.0,93471.0,93471.0,0.179799,-0.000021
2024-12-31 23:56:00,93469.0,93469.0,93469.0,93469.0,0.021400,-0.000449
2024-12-31 23:57:00,93462.0,93462.0,93413.0,93427.0,0.034750,0.000000


### hourly price data

In [19]:
df_hour = df.copy()

df_hour = df_hour.resample('h').agg({
  'Open': 'first',
  'High': 'max',
  'Low': 'min',
  'Close': 'last',
  'Volume': 'sum'
})

# calculate return (forward 1 hour)
df_hour['return_forward'] = df_hour['Close'].pct_change().shift(-1)

# last row does not have return_forward
df_hour = df_hour.iloc[:-1]

df_hour = df_hour[df_hour.index.year < 2025]

# save as parquet
df_hour.to_parquet('btcusd_hourly_price.parquet')

print(df_hour.shape)
df_hour.head()

(113966, 6)


C:\Users\phku0\AppData\Local\Temp\ipykernel_22868\430692146.py:12: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_hour['return_forward'] = df_hour['Close'].pct_change().shift(-1)


,Open,High,Low,Close,Volume,return_forward
Timestamp,,,,,,
2012-01-01 10:00:00,4.58,4.58,4.58,4.58,0.0,0.0
2012-01-01 11:00:00,4.58,4.58,4.58,4.58,0.0,0.0
2012-01-01 12:00:00,4.58,4.58,4.58,4.58,0.0,0.0
2012-01-01 13:00:00,4.58,4.58,4.58,4.58,0.0,0.0
2012-01-01 14:00:00,4.58,4.58,4.58,4.58,0.0,0.0


In [20]:
# print rows per year
print(df_hour.resample('YE').size())

Timestamp
2012-12-31    8774
2013-12-31    8760
2014-12-31    8760
2015-12-31    8760
2016-12-31    8784
2017-12-31    8760
2018-12-31    8760
2019-12-31    8760
2020-12-31    8784
2021-12-31    8760
2022-12-31    8760
2023-12-31    8760
2024-12-31    8784
Freq: YE-DEC, dtype: int64


### daily price data

In [21]:
df_day = df.copy()

df_day = df_day.resample('1D').agg({
  'Open': 'first',
  'High': 'max',
  'Low': 'min',
  'Close': 'last',
  'Volume': 'sum'
})

# calculate return (forward 1 day)
df_day['return_forward'] = df_day['Close'].pct_change().shift(-1)

# last row does not have return_forward
df_day = df_day.iloc[:-1]

df_day = df_day[df_day.index.year < 2025]

# save as parquet
df_day.to_parquet('btcusd_daily_price.parquet')

print(df_day.shape)
df_day

(4749, 6)


,Open,High,Low,Close,Volume,return_forward
Timestamp,,,,,,
2012-01-01,4.58,4.84,4.58,4.84,10.000000,0.033058
2012-01-02,4.84,5.00,4.84,5.00,10.100000,0.058000
2012-01-03,5.00,5.32,5.00,5.29,107.085281,0.052930
2012-01-04,5.29,5.57,4.93,5.57,107.233260,0.152603
2012-01-05,5.57,6.46,5.57,6.42,70.328742,-0.003115
...,...,...,...,...,...,...
2024-12-27,95606.00,97337.00,93282.00,94168.00,2060.503361,-0.000212
2024-12-28,94159.00,94159.00,94118.00,94148.00,0.015052,-0.006277
2024-12-29,95021.00,95124.00,92868.00,93557.00,762.959106,-0.000556
